In [1]:
import numpy as np
import pandas as pd
import nest
from scripts import optimization, neuron_categorization, visualizations


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.6.0
 Built: Sep 28 2023 11:53:34

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [2]:
nest.rng_seed = 1
np.random.seed(1)
nest.set_verbosity('M_FATAL')

In [3]:
calcium_data = np.load('/hpc/mzhu843/modelling/nest/DATA/processed/calcium_traces/C_3_p.npy')

spike_data = np.load('/hpc/mzhu843/modelling/nest/DATA/processed/spikes/S_3_p.npy')

ezTrack_data = pd.read_csv('/hpc/mzhu843/modelling/nest/DATA/raw/ezytrack/behaviour_LocationOutput.csv')


categorizer = neuron_categorization.NeuronCategorizer(spike_data, calcium_data, ezTrack_data)
categorizer.run_categorization()
categorized_neurons = categorizer.get_categorized_neurons()

selected_ids = ['30', '31', '51', '74', '77']
selected_cells = []
for id in selected_ids:
    selected_cells.append(categorized_neurons['Place'][id])
small_categorized_neurons = {'Place': selected_cells}

id_mask = [list(categorized_neurons['Place'].keys()).index(id) for id in selected_ids]

place_obs = categorizer.place_obs[id_mask][:, :2999]

initial_connectivity_matrix = np.load('/hpc/mzhu843/modelling/nest/DATA/raw/inital_matrices/initial_weights_2.npy')
initial_spikes_weights = np.ones((5, 3000))

lamb = 0




dict_keys(['2', '6', '7', '9', '12', '14', '15', '16', '20', '27', '28', '30', '31', '35', '36', '39', '45', '48', '51', '56', '61', '62', '67', '68', '69', '71', '73', '74', '75', '76', '77', '78', '79', '81', '84', '87', '90', '92', '93', '95', '97', '98', '100', '103', '105', '109', '110', '113', '114', '117', '118', '120', '124', '125', '126', '131', '133', '135', '136', '138', '141', '143', '144', '147', '150', '153', '154', '156', '160', '161', '162', '167', '171', '172', '174', '176', '177', '178', '179', '180', '181', '182', '184', '185', '186', '188', '189', '190', '191', '196', '199', '200'])


In [4]:
problem = optimization.SimulatedAnnealing2(initial_connectivity_matrix, place_obs , lamb, small_categorized_neurons, initial_spikes_weights)
problem.steps = 30000
x, func = problem.anneal()

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
  3962.23298     220273.47     0.00%     0.00%     0:00:41     0:02:46